In [ ]:
import os
import coacd
import trimesh

input_file = "<OBJECT-NAME>.obj"

mesh = trimesh.load(input_file, force="mesh")
mesh = coacd.Mesh(mesh.vertices, mesh.faces)
parts = coacd.run_coacd(mesh) # a list of convex hulls.

In [ ]:
import shutil
output_dir = input_file.replace(".obj", "")
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir)

rgba = "0.5 0.5 0.5 1"

asset_name = input_file.split("/")[-1].replace(".obj", "")

asset_config = """
<asset>
"""

body_config = """
<body name="{}" pos="0 0 0">
  <joint type="free"/>
""".format(asset_name)

for i, part in enumerate(parts):
    output_file = os.path.join(output_dir, f"part_{i}.obj")
    part_mesh = trimesh.Trimesh(vertices=part[0], faces=part[1])
    part_mesh.export(output_file)
    asset_config += '  <mesh name="{}_part_{}" file="obj/{}/part_{}.obj" scale="1 1 1"/>\n'.format(asset_name, i, asset_name, i)
    body_config += '  <geom type="mesh" rgba="{}" mesh="{}_part_{}"'.format(rgba, asset_name, i)
    body_config += ' class="obj_collision"/>\n'

asset_config += '</asset>\n'
print(asset_config)

body_config += """</body>\n"""
print(body_config)

